In [ ]:
class MLP_Model(nn.Module):
  def __init__(self,name="MLP_MODEL", vocab_size=260, embedding_dim=4, num_classes=4):
    super().__init__()
    self.name = name
    self.embedding = nn.Embedding(num_embeddings=vocab_size,embedding_dim=embedding_dim)
    # Input -> 256*192
    self.conv1 = nn.Conv2d(3, 128, 4, stride=2, padding=1, bias=False)
    # 256*192 -> conv1(1/2) -> 128*96
    self.bn1 = nn.BatchNorm2d(128)
    self.conv2 = nn.Conv2d(128, 256, 4, stride=2, padding=1, bias=False)
    # 128*96 -> conv1(1/2) -> 64*48
    self.bn2 = nn.BatchNorm2d(256)
    self.conv3 = nn.Conv2d(256, 128, 4, stride=2, padding=1, bias=False)
    # 64*48 -> conv1(1/2) -> 32*24
    self.bn3 = nn.BatchNorm2d(128)
    self.conv4 = nn.Conv2d(128, 32, 4, stride=2, padding=1, bias=False)
    # 32*24 -> conv1(1/2) -> 16*12
    self.bn4 = nn.BatchNorm2d(32)
    self.conv5 = nn.Conv2d(32, 8, 4, stride=2, padding=1, bias=False)
    # 16*12 -> conv1(1/2) -> 8*6
    self.bn5 = nn.BatchNorm2d(8)
    self.conv6 = nn.Conv2d(8, 1, 4, stride=2, padding=1, bias=False)
    # 8*6 -> conv1(1/2) -> 4*3
    self.bn6 = nn.BatchNorm2d(1)
    self.linear1 = nn.Linear(1*4*3+5+(embedding_dim-1), 1024)
    self.linear2 = nn.Linear(1024, 512)
    self.linear3 = nn.Linear(512, 128)
    self.linear4 = nn.Linear(128, 64)
    self.out = nn.Linear(64, num_classes)


  def forward(self, x,tabulars,debug = False):
    # entrada de 256*192
    emb_Location = self.embedding(tabulars[:,4])
    # area data scaling
    rescalArea = tabulars[:,0:1] /100.0
    #plt.imshow(x[0])
    x = x.view(x.size(0), 3, 256, 192)
    x = torch.round(x).to(torch.float32)
    x = F.relu(self.bn1(self.conv1(x)))
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.relu(self.bn3(self.conv3(x)))
    x = F.relu(self.bn4(self.conv4(x)))
    x = F.relu(self.bn5(self.conv5(x)))
    x = F.relu(self.bn6(self.conv6(x)))
    x = x.view(x.size(0), -1)
    x = torch.concat([x,rescalArea,tabulars[:,1:4], emb_Location], -1)
    if debug:
      print("Tabulars START:")
      print(x)
      print("Tabulars END:")
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.linear3(x))
    x = F.relu(self.linear4(x))
    x = self.out(x)
    x = F.softmax(x,dim = 1)
    return x